In [5]:
import pandas as pd
import numpy as np
import math
import time

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [7]:
train = pd.read_csv('../../../data/training.csv')
test = pd.read_csv('../../../data/testing.csv')
validation = pd.read_csv('../../../data/validation.csv')
vic_data = pd.read_csv('../../../data/victoria.csv')

In [8]:
# Constant Variables 
TIME = 'Time (UTC+10)'
PRICE = 'Regions VIC Trading Price ($/MWh)'
GENERATION = 'Regions VIC Trading Total Intermittent Generation (MW)'
DEMAND = 'Regions VIC Operational Demand (MW)'

POWER = 300
CAPACITY = 580
EFFICIENCY = 0.9
CHARGE_EFF = 90
DISCHARGE_EFF = 90
MLF = 0.991
FIXED_OP = 8.1
VAR_OP = 0

CHARGE = -1
DISCHARGE = 1

In [9]:
# percentile.exc from excel != np.percentile (np.percentile == percentile.inc from excel)
# code taken from https://stackoverflow.com/questions/38596100/python-equivalent-of-excels-percentile-exc

def quantile_exc(ser, q):
    ser_sorted = ser.sort_values()
    rank = q * (len(ser) + 1) - 1
    assert rank > 0, 'quantile is too small'
    rank_l = int(rank)
    return ser_sorted.iat[rank_l] + (ser_sorted.iat[rank_l + 1] - ser_sorted.iat[rank_l]) * (rank - rank_l)

In [10]:
def create_df(ori_df):
    """ Returns a proper dataframe with columns needed, along with the unused first row """

    df = ori_df[[TIME, PRICE]]
    df['raw_power'] = 0.0
    df['dispatch'] = 0.0
    df['revenue'] = 0.0
    df['opening'] = 0.0
    df['closing'] = 0.0
    df['decision1'] = 0
    
    # I removed the first row because first row of vic is 00:00:00, 
    # which is the last period from the previous year
    first_row = df.loc[0]
    df = df.drop([0], axis=0) 
    
    return df, first_row

In [19]:
def algo3(df):
    """ Finds optimal charge and discharge time across the dataset """
    
    for i in list(df.index):

        if ((i+LOOKAHEAD) < len(df)):

            thelist = df.iloc[i:i+LOOKAHEAD][PRICE]
            ser = pd.Series(thelist)
            LOWER_PERCENTILE = quantile_exc(ser,CHARGING_PERCENTILE)
            HIGHER_PERCENTILE = quantile_exc(ser,DISCHARGING_PERCENTILE)
            
            # charge if the current price is lower than the percentile 
            # and is lower than the mean of the next prices
            # discharge otherwise
            
            if (df.at[i,PRICE] <= LOWER_PERCENTILE):
                df.at[i,'decision1'] = CHARGE

            if (df.at[i,PRICE] >= HIGHER_PERCENTILE):
                df.at[i,'decision1'] = DISCHARGE
            
    return df

In [12]:
def get_opencap(i, df):
    """Get the opening battery capacity for every 30-minutes interval 
    Input:
        i : Current row in df
        df : DataFrame with 'opening' and 'closing' column
    Return 
        opening_cap : Opening battery capacity. Integer dtype
    
    """

    if i != 1: #Not the first row
        df.at[i,"opening"] = df.at[i-1,"closing"]
    opening_cap = df.at[i, "opening"]
    
    return opening_cap
    
def get_dispatch(rawPower):
    """ Get the power dispatched for every 30-minutes interval 
    Input:
        rawPower : Current rawPower. Integer Datatype
    Return:
        dispatch : Raw_power dispatched from the market. Integer dtype
    """
    if rawPower < 0:
        eff = 1
    else:
        eff = DISCHARGE_EFF / 100
            
    dispatch = (rawPower / 2) * eff
    return dispatch

def get_closecap(opening_cap, dispatch):
    """Get the battery closing capacity for every 30-minutes interval
    Input:
        opening_cap : Opening battery capacity. Integer dtype
        dispatch : Raw_power dispatched from the market. Integer dtype
    Return:
        closecap : Closing battery capacity. Integer dtype
    """
    if dispatch < 0:
        eff = CHARGE_EFF / 100
    else:
        eff = 100 / DISCHARGE_EFF

    closecap = round(max(0, min((opening_cap - (dispatch * eff)), CAPACITY)), 0)
    return closecap
    
def get_revenue(price, dispatch):
    """ Get the Revenue for every 30-minutes interval
    Input:
        price : Market spot price for electricity. Float dtype
        dispatch : Raw_power dispatched from the market. Integer dtype
    Return:
        revenue : current revenue. Integer dtype
    """
    if dispatch < 0:
        factor = 1/MLF
    else:
        factor = MLF 
    revenue = round(price * dispatch * factor)
    return revenue

def get_power(i, df, opening_cap):
    """ Get the Raw Power for every 30-minutes interval
    Input:
        i : Current row in df
        df : DataFrame with 'charge_forecaset' and 'discharge_forecast' column
    Return:
        opening_cap : Opening battery capacity. Integer dtype
    """
    raw_power = 0
    if (df.at[i,'decision1'] == CHARGE):
        raw_power = -min(POWER, (CAPACITY-opening_cap)/EFFICIENCY*2)   # FROM checkpoint3
        
    elif (df.at[i,'decision1'] == DISCHARGE):
        raw_power = min(POWER, opening_cap*2)      # FROM Algorithm 2 and 3 and checkpoint3
            
    return raw_power

def calculate(df):
    """ Calculate the Battery Opening, Closing Capacity, Raw Power, Market dispatch 
        and Revenue for the entire df.
    Input:
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column
    Return :
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column
    """
    # Go through each 30-minute interval of df
    for i in list(df.index):
        # get current Spot Price
        price = df.at[i, PRICE]
        
        # update opening capacity
        opening_cap = get_opencap(i, df)

        # find raw_power
        rawPower = get_power(i, df, opening_cap)
        df.at[i,'raw_power'] = rawPower

        # find market_dispatch 
        dispatch = get_dispatch(rawPower)
        df.at[i,"dispatch"] = dispatch

        # find closing_capacity   
        df.at[i,"closing"] = get_closecap(opening_cap, dispatch)

        #find revenue        
        df.at[i,"revenue"] = get_revenue(price, dispatch)
    
    return df

In [15]:
def show_result(df):
    """ Print the revenue related information computed from df.
    Input: 
        df : DataFrame with 'revenue' column.
    """
    print("Total revenue in the dataset:", df["revenue"].sum())
    print("Total days in the dataset:", len(df)/48)
    print("Revenue per day:", df["revenue"].sum() / (len(df)/48))
    
    return None

def run_all(ori_df):
    """ Run the entire data pipeline including initialisng (Data Processing), 
    finding the optimal charging and discarging period (Data Modelling), 
    calculating the revenue based on the optimal period mentioned above (Model Testing and Evaluation).
    
    Input:
        ori_df : DataFrame which contains spot_price for every 30-minute interval. 
    Return:
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column.
    """
    # Start time
    start = time.time()
    # Initialise df
    df2, first_row = create_df(ori_df)
    
    # Find Optimal Charging and Discharging period
    df3 = algo3(df2)
    
    # Calculate the revenue
    df = calculate(df3)
    
    # Add unused row back to df
    df.loc[0] = first_row
    df = df.sort_index()
    
    # Show the revenue       
    show_result(df)
    # End Time      
    end = time.time()
    print("Time Complexity for running the entire Algorithm 3: {time_taken}s".format(time_taken = end-start))
            
    return df

In [21]:
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.25
DISCHARGING_PERCENTILE = 0.75
vic = run_all(vic_data)

Total revenue in the dataset: 118104744.0
Total days in the dataset: 1322.0208333333333
Revenue per day: 89336.52255858298
Time Complexity for running the entire Algorithm 3: 20.693780660629272s
